In [1]:
# Imports
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import csv
from urllib.parse import urljoin


In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36',
}


In [3]:
r = requests.get("https://arxiv.org/list/cs/pastweek?skip=0&show=25")

In [4]:
soup = BeautifulSoup(r.text,'lxml')

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [5]:
identifier_list = soup.find_all('span', class_='list-identifier')

In [6]:
papers_info = []


In [7]:
# import requests
# from time import sleep
# from requests.exceptions import Timeout, RequestException

# # 定义最大重试次数
# max_retries = 3

# for identifier in identifier_list:
#     link_list = identifier.find('a', {'title': 'Abstract'})
#     abs_url = link_list['href']
#     abs_full_url = urljoin("https://arxiv.org", abs_url)
    
#     # 初始化重试次数
#     retry_count = 0
#     while retry_count < max_retries:
#         try:
#             response = requests.get(abs_full_url, timeout=20)  
#             sleep(2)  # 在每次请求之后等待2秒钟
#             response.raise_for_status()  # 检查请求是否成功
#             break  # 请求成功，跳出循环
#         except (Timeout, RequestException) as e:
#             print(f"Request error: {e}")
#             retry_count += 1
#             if retry_count == max_retries:
#                 print("Max retries reached, unable to fetch data")
#                 break  # 达到最大重试次数，跳出循环
#             else:
#                 print(f"Retrying... Attempt {retry_count}")
#                 sleep(2)  # 在重试之前等待一段时间
    
#     if retry_count < max_retries:  # 如果请求成功
#         soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
#         abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
#         title = soup1.find('h1', class_='title mathjax').text.strip()
#         authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
#         subjects = soup1.find('span', class_='primary-subject').text.strip()
#         paper_info = {
#             'Title': title,
#             'Subjects': subjects,
#             'Authors': authors,
#             'Abstract': abstract
#         }
#         papers_info.append(paper_info)
#         print(abstract)  # Print or do whatever you need with the abstract

import requests
from time import sleep
from requests.exceptions import Timeout, RequestException

# 定义最大重试次数
max_retries = 5
# 存储失败的请求
failed_requests = []
for identifier  in identifier_list :
    link_list = identifier.find('a', {'title': 'Abstract'})
    abs_url = link_list['href']
    abs_full_url = urljoin("https://arxiv.org", abs_url)
    
    # 初始化重试次数
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.get(abs_full_url, timeout=20)  # 设置超时时间为10秒
            sleep(2)  # 在每次请求之后等待2秒钟
            response.raise_for_status()  # 检查请求是否成功
            break  # 请求成功，跳出循环
        except (Timeout, RequestException) as e:
            print(f"Request error: {e}")
            retry_count += 1
            if retry_count == max_retries:
                print("Max retries reached, unable to fetch data")
                # 将失败的请求记录下来
                failed_requests.append(abs_full_url)
                break  # 达到最大重试次数，跳出循环
            else:
                print(f"Retrying... Attempt {retry_count}")
                sleep(2)  # 在重试之前等待一段时间
    
    if retry_count < max_retries:  # 如果请求成功
        soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
        abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
        title = soup1.find('h1', class_='title mathjax').text.strip()
        authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
        subjects = soup1.find('span', class_='primary-subject').text.strip()
        paper_info = {
            'Title': title,
            'Subjects': subjects,
            'Authors': authors,
            'Abstract': abstract
        }
        papers_info.append(paper_info)
        print(abstract)  # Print or do whatever you need with the abstract

if failed_requests:  # 检查 failed_requests 是否为空
    for failed_request in failed_requests:
        try:
            response = requests.get(failed_request, timeout=20)
            sleep(2)  # 在每次请求之后等待2秒钟
            response.raise_for_status()  # 检查请求是否成功
            break  # 请求成功，跳出循环
        except (Timeout, RequestException) as e:
            print(f"Request error: {e}")
            retry_count += 1
            if retry_count == max_retries:
                print("Max retries reached, unable to fetch data")
                # 将失败的请求记录下来
                failed_requests.append(abs_full_url)
                break  # 达到最大重试次数，跳出循环
            else:
                print(f"Retrying... Attempt {retry_count}")
                sleep(2)  # 在重试之前等待一段时间
    
    if retry_count < max_retries:  # 如果请求成功
        soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
        abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
        title = soup1.find('h1', class_='title mathjax').text.strip()
        authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
        subjects = soup1.find('span', class_='primary-subject').text.strip()
        paper_info = {
            'Title': title,
            'Subjects': subjects,
            'Authors': authors,
            'Abstract': abstract
        }
        papers_info.append(paper_info)
        print(abstract) 
else:
    print("No failed requests to retry.")


Abstract:Many online content portals allow users to ask questions to supplement their understanding (e.g., of lectures). While information retrieval (IR) systems may provide answers for such user queries, they do not directly assist content creators -- such as lecturers who want to improve their content -- identify segments that _caused_ a user to ask those questions. We introduce the task of backtracing, in which systems retrieve the text segment that most likely caused a user query. We formalize three real-world domains for which backtracing is important in improving content delivery and communication: understanding the cause of (a) student confusion in the Lecture domain, (b) reader curiosity in the News Article domain, and (c) user emotion in the Conversation domain. We evaluate the zero-shot performance of popular information retrieval methods and language modeling methods, including bi-encoder, re-ranking and likelihood-based methods and ChatGPT. While traditional IR systems retr

In [8]:
r1 = requests.get("https://arxiv.org/list/cs/pastweek?skip=25&show=25")

In [9]:
soup1 = BeautifulSoup(r1.text,'lxml')

In [10]:
identifier_list1 = soup1.find_all('span', class_='list-identifier')

In [11]:
# # 定义最大重试次数
# max_retries = 3

# for identifier1 in identifier_list1:
#     link_list = identifier1.find('a', {'title': 'Abstract'})
#     abs_url = link_list['href']
#     abs_full_url = urljoin("https://arxiv.org", abs_url)
    
#     # 初始化重试次数
#     retry_count = 0
#     while retry_count < max_retries:
#         try:
#             response = requests.get(abs_full_url, timeout=20)  
#             sleep(2)  # 在每次请求之后等待2秒钟
#             response.raise_for_status()  # 检查请求是否成功
#             break  # 请求成功，跳出循环
#         except (Timeout, RequestException) as e:
#             print(f"Request error: {e}")
#             retry_count += 1
#             if retry_count == max_retries:
#                 print("Max retries reached, unable to fetch data")
#                 break  # 达到最大重试次数，跳出循环
#             else:
#                 print(f"Retrying... Attempt {retry_count}")
#                 sleep(2)  # 在重试之前等待一段时间
    
#     if retry_count < max_retries:  # 如果请求成功
#         soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
#         abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
#         title = soup1.find('h1', class_='title mathjax').text.strip()
#         authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
#         subjects = soup1.find('span', class_='primary-subject').text.strip()
#         paper_info = {
#             'Title': title,
#             'Subjects': subjects,
#             'Authors': authors,
#             'Abstract': abstract
#         }
#         papers_info.append(paper_info)
#         print(abstract)  # Print or do whatever you need with the abstract


import requests
from time import sleep
from requests.exceptions import Timeout, RequestException
# 存储失败的请求
failed_requests = []
# 定义最大重试次数
max_retries = 5
for identifier1 in identifier_list1:
    link_list = identifier1.find('a', {'title': 'Abstract'})
    abs_url = link_list['href']
    abs_full_url = urljoin("https://arxiv.org", abs_url)
    
    # 初始化重试次数
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.get(abs_full_url, timeout=20)  # 设置超时时间为10秒
            sleep(2)  # 在每次请求之后等待2秒钟
            response.raise_for_status()  # 检查请求是否成功
            break  # 请求成功，跳出循环
        except (Timeout, RequestException) as e:
            print(f"Request error: {e}")
            retry_count += 1
            if retry_count == max_retries:
                print("Max retries reached, unable to fetch data")
                # 将失败的请求记录下来
                failed_requests.append(abs_full_url)
                break  # 达到最大重试次数，跳出循环
            else:
                print(f"Retrying... Attempt {retry_count}")
                sleep(2)  # 在重试之前等待一段时间
    
    if retry_count < max_retries:  # 如果请求成功
        soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
        abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
        title = soup1.find('h1', class_='title mathjax').text.strip()
        authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
        subjects = soup1.find('span', class_='primary-subject').text.strip()
        paper_info = {
            'Title': title,
            'Subjects': subjects,
            'Authors': authors,
            'Abstract': abstract
        }
        papers_info.append(paper_info)
        print(abstract)  # Print or do whatever you need with the abstract

if failed_requests:  # 检查 failed_requests 是否为空
    for failed_request in failed_requests:
        try:
            response = requests.get(failed_request, timeout=20)
            sleep(2)  # 在每次请求之后等待2秒钟
            response.raise_for_status()  # 检查请求是否成功
            break  # 请求成功，跳出循环
        except (Timeout, RequestException) as e:
            print(f"Request error: {e}")
            retry_count += 1
            if retry_count == max_retries:
                print("Max retries reached, unable to fetch data")
                # 将失败的请求记录下来
                failed_requests.append(abs_full_url)
                break  # 达到最大重试次数，跳出循环
            else:
                print(f"Retrying... Attempt {retry_count}")
                sleep(2)  # 在重试之前等待一段时间
    
    if retry_count < max_retries:  # 如果请求成功
        soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
        abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
        title = soup1.find('h1', class_='title mathjax').text.strip()
        authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
        subjects = soup1.find('span', class_='primary-subject').text.strip()
        paper_info = {
            'Title': title,
            'Subjects': subjects,
            'Authors': authors,
            'Abstract': abstract
        }
        papers_info.append(paper_info)
        print(abstract) 
else:
    print("No failed requests to retry.")

Request error: HTTPSConnectionPool(host='arxiv.org', port=443): Read timed out. (read timeout=20)
Retrying... Attempt 1
Abstract:Simulation is an invaluable tool for radio-frequency system designers that enables rapid prototyping of various algorithms for imaging, target detection, classification, and tracking. However, simulating realistic radar scans is a challenging task that requires an accurate model of the scene, radio frequency material properties, and a corresponding radar synthesis function. Rather than specifying these models explicitly, we propose DART - Doppler Aided Radar Tomography, a Neural Radiance Field-inspired method which uses radar-specific physics to create a reflectance and transmittance-based rendering pipeline for range-Doppler images. We then evaluate DART by constructing a custom data collection platform and collecting a novel radar dataset together with accurate position and instantaneous velocity measurements from lidar-based localization. In comparison to 

In [13]:
r2 = requests.get("https://arxiv.org/list/cs/pastweek?skip=50&show=25")

In [14]:
soup2 = BeautifulSoup(r2.text,'lxml')

In [15]:
identifier_list2 = soup2.find_all('span', class_='list-identifier')

In [16]:
# # 定义最大重试次数
# max_retries = 3

# for identifier2 in identifier_list2:
#     link_list = identifier2.find('a', {'title': 'Abstract'})
#     abs_url = link_list['href']
#     abs_full_url = urljoin("https://arxiv.org", abs_url)
    
#     # 初始化重试次数
#     retry_count = 0
#     while retry_count < max_retries:
#         try:
#             response = requests.get(abs_full_url, timeout=20)  
#             sleep(2)  # 在每次请求之后等待2秒钟
#             response.raise_for_status()  # 检查请求是否成功
#             break  # 请求成功，跳出循环
#         except (Timeout, RequestException) as e:
#             print(f"Request error: {e}")
#             retry_count += 1
#             if retry_count == max_retries:
#                 print("Max retries reached, unable to fetch data")
#                 break  # 达到最大重试次数，跳出循环
#             else:
#                 print(f"Retrying... Attempt {retry_count}")
#                 sleep(2)  # 在重试之前等待一段时间
    
#     if retry_count < max_retries:  # 如果请求成功
#         soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
#         abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
#         title = soup1.find('h1', class_='title mathjax').text.strip()
#         authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
#         subjects = soup1.find('span', class_='primary-subject').text.strip()
#         paper_info = {
#             'Title': title,
#             'Subjects': subjects,
#             'Authors': authors,
#             'Abstract': abstract
#         }
#         papers_info.append(paper_info)
#         print(abstract)  # Print or do whatever you need with the abstract

import requests
from time import sleep
from requests.exceptions import Timeout, RequestException
# 存储失败的请求
failed_requests = []
# 定义最大重试次数
max_retries = 5
for identifier2 in identifier_list2:
    link_list = identifier2.find('a', {'title': 'Abstract'})
    abs_url = link_list['href']
    abs_full_url = urljoin("https://arxiv.org", abs_url)
    
    # 初始化重试次数
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.get(abs_full_url, timeout=20)  # 设置超时时间为10秒
            sleep(2)  # 在每次请求之后等待2秒钟
            response.raise_for_status()  # 检查请求是否成功
            break  # 请求成功，跳出循环
        except (Timeout, RequestException) as e:
            print(f"Request error: {e}")
            retry_count += 1
            if retry_count == max_retries:
                print("Max retries reached, unable to fetch data")
                # 将失败的请求记录下来
                failed_requests.append(abs_full_url)
                break  # 达到最大重试次数，跳出循环
            else:
                print(f"Retrying... Attempt {retry_count}")
                sleep(2)  # 在重试之前等待一段时间
    
    if retry_count < max_retries:  # 如果请求成功
        soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
        abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
        title = soup1.find('h1', class_='title mathjax').text.strip()
        authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
        subjects = soup1.find('span', class_='primary-subject').text.strip()
        paper_info = {
            'Title': title,
            'Subjects': subjects,
            'Authors': authors,
            'Abstract': abstract
        }
        papers_info.append(paper_info)
        print(abstract)  # Print or do whatever you need with the abstract

if failed_requests:  # 检查 failed_requests 是否为空
    for failed_request in failed_requests:
        try:
            response = requests.get(failed_request, timeout=20)
            sleep(2)  # 在每次请求之后等待2秒钟
            response.raise_for_status()  # 检查请求是否成功
            break  # 请求成功，跳出循环
        except (Timeout, RequestException) as e:
            print(f"Request error: {e}")
            retry_count += 1
            if retry_count == max_retries:
                print("Max retries reached, unable to fetch data")
                # 将失败的请求记录下来
                failed_requests.append(abs_full_url)
                break  # 达到最大重试次数，跳出循环
            else:
                print(f"Retrying... Attempt {retry_count}")
                sleep(2)  # 在重试之前等待一段时间
    
    if retry_count < max_retries:  # 如果请求成功
        soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
        abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
        title = soup1.find('h1', class_='title mathjax').text.strip()
        authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
        subjects = soup1.find('span', class_='primary-subject').text.strip()
        paper_info = {
            'Title': title,
            'Subjects': subjects,
            'Authors': authors,
            'Abstract': abstract
        }
        papers_info.append(paper_info)
        print(abstract) 
else:
    print("No failed requests to retry.")

Abstract:Emoji have become ubiquitous in written communication, on the Web and beyond. They can emphasize or clarify emotions, add details to conversations, or simply serve decorative purposes. This casual use, however, barely scratches the surface of the expressive power of emoji. To further unleash this power, we present Emojinize, a method for translating arbitrary text phrases into sequences of one or more emoji without requiring human input. By leveraging the power of large language models, Emojinize can choose appropriate emoji by disambiguating based on context (eg, cricket-bat vs bat) and can express complex concepts compositionally by combining multiple emoji (eq, ''Emojinize'' is translated to input-latin-letters right-arrow grinning-face). In a cloze test--based user study, we show that Emojinize's emoji translations increase the human guessability of masked words by 55%, whereas human-picked emoji translations do so by only 29%. These results suggest that emoji provide a su

In [17]:
r3 = requests.get("https://arxiv.org/list/cs/pastweek?skip=75&show=25")

In [18]:
soup3 = BeautifulSoup(r3.text,'lxml')

In [19]:
identifier_list3 = soup3.find_all('span', class_='list-identifier')

In [20]:
# # 定义最大重试次数
# max_retries = 3

# for identifier3 in identifier_list3:
#     link_list = identifier3.find('a', {'title': 'Abstract'})
#     abs_url = link_list['href']
#     abs_full_url = urljoin("https://arxiv.org", abs_url)
    
#     # 初始化重试次数
#     retry_count = 0
#     while retry_count < max_retries:
#         try:
#             response = requests.get(abs_full_url, timeout=20)  
#             sleep(2)  # 在每次请求之后等待2秒钟
#             response.raise_for_status()  # 检查请求是否成功
#             break  # 请求成功，跳出循环
#         except (Timeout, RequestException) as e:
#             print(f"Request error: {e}")
#             retry_count += 1
#             if retry_count == max_retries:
#                 print("Max retries reached, unable to fetch data")
#                 break  # 达到最大重试次数，跳出循环
#             else:
#                 print(f"Retrying... Attempt {retry_count}")
#                 sleep(2)  # 在重试之前等待一段时间
    
#     if retry_count < max_retries:  # 如果请求成功
#         soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
#         abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
#         title = soup1.find('h1', class_='title mathjax').text.strip()
#         authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
#         subjects = soup1.find('span', class_='primary-subject').text.strip()
#         paper_info = {
#             'Title': title,
#             'Subjects': subjects,
#             'Authors': authors,
#             'Abstract': abstract
#         }
#         papers_info.append(paper_info)
#         print(abstract)  # Print or do whatever you need with the abstract



import requests
from time import sleep
from requests.exceptions import Timeout, RequestException
# 存储失败的请求
failed_requests = []
# 定义最大重试次数
max_retries = 5
for identifier3 in identifier_list3:
    link_list = identifier3.find('a', {'title': 'Abstract'})
    abs_url = link_list['href']
    abs_full_url = urljoin("https://arxiv.org", abs_url)
    
    # 初始化重试次数
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.get(abs_full_url, timeout=20)  # 设置超时时间为10秒
            sleep(2)  # 在每次请求之后等待2秒钟
            response.raise_for_status()  # 检查请求是否成功
            break  # 请求成功，跳出循环
        except (Timeout, RequestException) as e:
            print(f"Request error: {e}")
            retry_count += 1
            if retry_count == max_retries:
                print("Max retries reached, unable to fetch data")
                # 将失败的请求记录下来
                failed_requests.append(abs_full_url)
                break  # 达到最大重试次数，跳出循环
            else:
                print(f"Retrying... Attempt {retry_count}")
                sleep(2)  # 在重试之前等待一段时间
    
    if retry_count < max_retries:  # 如果请求成功
        soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
        abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
        title = soup1.find('h1', class_='title mathjax').text.strip()
        authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
        subjects = soup1.find('span', class_='primary-subject').text.strip()
        paper_info = {
            'Title': title,
            'Subjects': subjects,
            'Authors': authors,
            'Abstract': abstract
        }
        papers_info.append(paper_info)
        print(abstract)  # Print or do whatever you need with the abstract

if failed_requests:  # 检查 failed_requests 是否为空
    for failed_request in failed_requests:
        try:
            response = requests.get(failed_request, timeout=20)
            sleep(2)  # 在每次请求之后等待2秒钟
            response.raise_for_status()  # 检查请求是否成功
            break  # 请求成功，跳出循环
        except (Timeout, RequestException) as e:
            print(f"Request error: {e}")
            retry_count += 1
            if retry_count == max_retries:
                print("Max retries reached, unable to fetch data")
                # 将失败的请求记录下来
                failed_requests.append(abs_full_url)
                break  # 达到最大重试次数，跳出循环
            else:
                print(f"Retrying... Attempt {retry_count}")
                sleep(2)  # 在重试之前等待一段时间
    
    if retry_count < max_retries:  # 如果请求成功
        soup1 = BeautifulSoup(response.text, 'html.parser')  # Specify parser explicitly
        abstract = soup1.find('blockquote', class_='abstract mathjax').text.strip()
        title = soup1.find('h1', class_='title mathjax').text.strip()
        authors = soup1.find('div', class_='authors').text.strip().replace(',', ';')
        subjects = soup1.find('span', class_='primary-subject').text.strip()
        paper_info = {
            'Title': title,
            'Subjects': subjects,
            'Authors': authors,
            'Abstract': abstract
        }
        papers_info.append(paper_info)
        print(abstract) 
else:
    print("No failed requests to retry.")

Abstract:We introduce a new family of prompt injection attacks, termed Neural Exec. Unlike known attacks that rely on handcrafted strings (e.g., "Ignore previous instructions and..."), we show that it is possible to conceptualize the creation of execution triggers as a differentiable search problem and use learning-based methods to autonomously generate them.
Our results demonstrate that a motivated adversary can forge triggers that are not only drastically more effective than current handcrafted ones but also exhibit inherent flexibility in shape, properties, and functionality. In this direction, we show that an attacker can design and generate Neural Execs capable of persisting through multi-stage preprocessing pipelines, such as in the case of Retrieval-Augmented Generation (RAG)-based applications. More critically, our findings show that attackers can produce triggers that deviate markedly in form and shape from any known attack, sidestepping existing blacklist-based detection and 

In [21]:
#papers_info.append(papers_info1)
with open('papers_info.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Title', 'Subjects', 'Authors','Abstract']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_MINIMAL)
    writer.writeheader()

    # 遍历每篇论文的信息，写入 CSV 文件
    for paper in papers_info:
        writer.writerow({'Title': paper.get('Title', ''),
                         'Subjects': paper.get('Subjects', ''),
                         'Authors': paper.get('Authors', ''),
                         'Abstract': paper.get('Abstract','') })